In [1]:
from pptx import Presentation
from bs4 import BeautifulSoup as bs
import markdown
import os
import re

In [2]:
# prs = Presentation("BaseballCard.pptx")
# # bbcard_entity_layout = prs.slide_layouts[7]
# # slide = prs.slides.add_slide(bbcard_entity_layout)
# # title = slide.shapes.title
# # # subtitle = slide.placeholders[1]

# # title.text = "Hello, World!"
# # subtitle.text = "python-pptx was here!"

# prs.save('BlankBBCard.pptx')

### Read in Obsidian files

In [84]:
class ProfileBuilder():
    def __init__(self):
        print("init()")
        
        self.path = None
        self.entities_to_parse = []
        self.raw_entity_data = {}
        self.final_entity_data = {}
        
    def create_profiles(self, path: str, entity_type: str):
        print("create_profiles()")
        self.path = path
        
        self.scan_dir_for_md_files()
        
        self.read_md_files_into_soup(entity_type)
        print(len(self.raw_entity_data))
        
        self.parse_entity_info_from_soup()
    
    def scan_dir_for_md_files(self):
#         entities_to_parse = []
        for file in os.listdir(self.path):
            if file.endswith("md"):
                self.entities_to_parse.append(file)
#         return entities_to_parse
    
    def read_md_files_into_soup(self, entity_type):
        for i in self.entities_to_parse[0:1]:
            name = i.replace(".md", "")
            self.raw_entity_data[name] = {"entity_type": entity_type}
            self.final_entity_data[name] = {"entity_type": entity_type}

            html = None
            with open(f"{self.path}/{i}", "r", encoding="utf-8") as f:
                text = f.read()

            html = markdown.markdown(text)
            soup = bs(html)
            self.raw_entity_data[name]["soup"] = soup
        
    def parse_entity_info_from_soup(self):
        for name, data in self.raw_entity_data.items():            
            # Change to switch statement !
            if data["entity_type"] == "person":
                print("this is a person profile")
                self.get_person_info(name, data["soup"])
    
    def get_person_info(self, name, soup):
        # Get all paragraph tags
        all_p_tags = soup.find_all("p")
        print(all_p_tags)
        
        # Find pic filename and create path string
        pic_path = self.get_person_pic_path(all_p_tags)
        print(pic_path)
        
        self.final_entity_data[name]["pic_path"] = pic_path
        print()
        
        # Find all list item tags
        all_li_tags = soup.find_all("li")
        print(all_li_tags)
        
        print(self.final_entity_data)


    def get_person_pic_path(self, all_p_tags):
        person_pic = None

        # Find correct p tag to parse
        try:
            person_pic = list(filter(lambda x: "Picture" in x.get_text(), all_p_tags))[0]
        except IndexError:
            print("There is no p tag with a picture in it")
            pass

        if not person_pic:
            return

        # Parse filename from p tag
        bracketed_filename_pattern = "([0-9A-Z]{1,}[\_\s]{0,}[0-9A-Z]{1,}\.{1}[0-9A-Z]{1,3})"
        try:
            person_pic = re.search(bracketed_filename_pattern, str(person_pic), re.IGNORECASE)[0]
        except TypeError:
            print("There is no pic filename")
            person_pic = None
            pass

        if not person_pic:
            return

        print(person_pic)

        return f"Assets/{person_pic.strip()}"

In [85]:
target_dir = "Investigation/People"

builder = ProfileBuilder()
builder.create_profiles(target_dir, "person")

init()
create_profiles()
1
this is a person profile
[<p>tags:
  - Person_Of_Interest</p>, <p><code>Picture</code> <br/>![[Arron_Gaines.jpg | 250]]</p>, <p><code>Detail</code> Subject likely lives and works in New York City, New York based on social media including pictures and check-ins. Follower/following relationships on social media platforms indicate a connection with Austin, Texas-based [[Kevin Ellison]]. Research on social engineering forums revealed usernames that were traced back to email addresses with a common web domain which has been separately flagged for malicious activity. Additional research on the emails suggests Gaines and Ellison are the probable end users. </p>]
Arron_Gaines.jpg
Assets/Arron_Gaines.jpg

[<li><code>Note</code> No previous criminal convictions</li>, <li><code>Note</code> Known phishing victims include: [[Elsie Graves]], [[Wilfredo Goodman]], and [[Jan Jackson]]</li>, <li><code>Associate</code> [[Kevin Ellison]]</li>, <li><code>Email Address</code> <em

In [19]:
# raw_entity_data = {}
# final_entity_data = {}

# # Read each markdown file in and convert to HTML then convert to soup
# for i in entities_to_parse[0:]:
#     name = i.replace(".md", "")
#     raw_entity_data[name] = {}
#     final_entity_data[name] = {}
    
#     html = None
#     with open(f"{case_dir}/{target_dir}/{i}", "r", encoding="utf-8") as f:
#         text = f.read()
    
#     html = markdown.markdown(text)
#     soup = bs(html)
#     raw_entity_data[name]["soup"] = soup

# # raw_entity_data

In [22]:
# # Determine if file is for Person of Interest
# to_find = "Person_Of_Interest"

# is_person_file = any([to_find in tag.get_text() for tag in all_p_tags])
# is_person_file

True

In [26]:
prefixes = {
    "Detail": [
        "Detail",
    ],
    "Associate": [
        "Associate",
    ],
    "Identifier": [
        "Identifier",
    ],
    "Location": [
        "Work Name",
        "Work Address",
        "Work Coordinates",
        "Residence Name",
        "Residence Address",
        "Residence Coordinates",
    ],
}

In [ ]:
def gather_li_tags_by_type(x, tag_types):
    code_tag = None
    try:
        code_tag = x.find("code")
    except TypeError:
        print("li did not contain a `Code` tag ")
        return False
    
    if not code_tag:
        return False
    
    if any(tag in code_tag for tag in tag_types):
        return True
    else:
        return False

In [ ]:
detail_tags = list(filter(lambda x: gather_li_tags_by_type(x, prefixes["Detail"]), all_li_tags))
detail_tags

In [ ]:
associate_tags = list(filter(lambda x: gather_li_tags_by_type(x, prefixes["Associate"]), all_li_tags))
associate_tags

In [ ]:
identifier_tags = list(filter(lambda x: gather_li_tags_by_type(x, prefixes["Identifier"]), all_li_tags))
identifier_tags

In [ ]:
location_tags = list(filter(lambda x: gather_li_tags_by_type(x, prefixes["Location"]), all_li_tags))
location_tags

In [ ]:
def remove_code_tag_substring(raw_tag):
    cleaned_tag = None
    try:
        cleaned_tag = re.sub("<code>.*</code>", "", str(raw_tag))
        return cleaned_tag
    except:
        print("substring doesn't have a `Code` tag in it")
        return raw_tag

def get_text_inside_li_tags(raw_tag):
    inside_text = None
    try:
        inside_text = re.match("(<li>)(.*)(</li>)", raw_tag)[2]
        return inside_text.strip()
    except:
        return raw_tag

def get_text_from_obsidian_links(raw_tag):
    """Match Obsidian's link format and extract characters between double square brackets"""
    try:
        match_obj = re.match("(.*)(\[\[)(.*)(\]\])(.*)", raw_tag)        
#         print("0", match_obj[0]) # match object
#         print("1", match_obj[1]) # match group 1, etc...
#         print("2", match_obj[2])
#         print("3", match_obj[3])
#         print("4", match_obj[4])
#         print("5", match_obj[5])
        text_before = match_obj[1]
        text_inside = match_obj[3]
        text_after = match_obj[5]
        rebuilt = f"{text_before.strip()} {text_inside.strip()} {text_after.strip()}"
        return rebuilt.strip()
    except TypeError:
        print("did not have any double brackets")
        return raw_tag.strip()

def extract_info_from_li_tag(x):
    cleaned = remove_code_tag_substring(x)
#     print("cleaned1: ", cleaned)
    
    cleaned = get_text_inside_li_tags(cleaned)
#     print("cleaned2: ", cleaned)

    cleaned = get_text_from_obsidian_links(cleaned)
#     print("cleaned3: ", cleaned)

    return cleaned

In [ ]:
detail_tags = list(map(lambda x: extract_info_from_li_tag(x), detail_tags))
detail_tags

In [ ]:
associate_tags = list(map(lambda x: extract_info_from_li_tag(x), associate_tags))
associate_tags

In [ ]:
identifier_tags = list(map(lambda x: extract_info_from_li_tag(x), identifier_tags))
identifier_tags

In [ ]:
location_tags = list(map(lambda x: extract_info_from_li_tag(x), location_tags))
location_tags